In [24]:
import numpy as np
import png
import os
import nibabel as nib
from matplotlib import pyplot as pl
import matplotlib
import math
import csv
import cv2
from tensorflow import keras
import tensorflow as tf
import sys

In [25]:
verbose = False

In [26]:
# input_folder = folder that contains the .nii file
# output_folder = folder that the .png file will be saved in
# model = a model deciding whether to include or discard brain layers
def nii_to_png(input_folder, output_folder, model):
    config = model.get_config() # Returns pretty much every information about your model
    input_shape = config["layers"][0]["config"]["batch_input_shape"][1:3] # width and height
    filenames = os.listdir(input_folder)
    for f in filenames:
        if f.endswith('.nii'):
            #Start reading nii files
            img_path = os.path.join(input_folder, f)
            img = nib.load(img_path) #read nii            
            img_fdata = img.get_fdata()
            fname = filename[ filename.rfind('_')+1 : filename.find('.nii') ]

            #  to image
            (x,y,z) = img.shape
            for i in range(0,x): #x is the sequence of images
                slice = img_fdata[i, :, :] #Select which direction the slice can be
                #slice_resized = cv2.resize(slice, dsize=input_shape, interpolation=cv2.INTER_LINEAR)
                #slice_resized = tf.convert_to_tensor( slice_resized.reshape(None, *input_shape, 3) )
                
                #if model.predict_step(slice_resized) == 0:
                matplotlib.image.imsave(os.path.join(output_folder,'{}-{}.png'.format(fname, i)), slice)

In [29]:
input_folder = 'C:\\Users\\dongh\\Desktop\\ADNI1_Complete 1Yr 1.5T (1)\\ADNI'
output_folder = 'C:\\Users\\dongh\\Desktop\\ADNI1_Complete 1Yr 1.5T (1)\\PNG'
metadata = 'C:\\Users\\dongh\\Downloads\\ADNI1_Complete_1Yr_1.5T_1_26_2022.csv'

# use the csv metadata to correctly categorize the patients into CN, MCI, AD
img_group_dict = {}
# image groups: cognitively normal, mild cognitive impairment, Alzheimer's disease
group_folder_dict = {'CN':'NonDemented', 'MCI':'Demented', 'AD':'Demented'}
with open(metadata, newline='') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',', quotechar='\"')
    for row in reader:
        # this dictionary will be used to find what group (output folder) 
        # a patient belongs to
        img_group_dict[ row['Image Data ID'] ] = row['Group']

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# create subfolders in the output folder
for group in group_folder_dict:
    subfolder = os.path.join(output_folder, group_folder_dict[group])
    # put the generated subfolder directory in the dictionary for image export purposes
    group_folder_dict[group] = subfolder
    if not os.path.exists(subfolder):
        os.makedirs(subfolder)


In [30]:
vgg19 = keras.models.load_model('Brain Layer Selector')

for root, dirnames, filenames in os.walk(input_folder):
    # no more subdirectories; we are looking at a folder with a .nii file
    if len(dirnames) == 0:
        filename = filenames[0] # there's only one file in the folder so just pick it
        img_id = filename[ filename.rfind('_')+1 : filename.find('.nii') ]
        group = img_group_dict[img_id]
        print(img_id, group, sep=', ')
        nii_to_png(root, group_folder_dict[group], vgg19)

I86020, CN
I40683, CN
I74174, CN
I118680, CN
I73881, CN
I81330, CN
I35499, MCI
I35569, MCI
I87484, MCI
I70648, MCI
I65167, MCI
I71606, MCI
I91985, MCI
I40312, AD
I40303, AD
I66745, AD
